# Support Vector Machines

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('../datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('../datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e Modelação

In [2]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

For support vector machines its highly recommended to perform normalization to improve model performance and convergence speed. 

In [3]:
scaler = StandardScaler()

X_treino_normalized = scaler.fit_transform(X_treino)
X_teste_normalized = scaler.fit_transform(X_teste)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

svm_model = svm.SVC(random_state=2023)

# gamma: [1, 0.1, 0.01, 0.001, 0.0001]

params = {
    'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['rbf']
}

grid = GridSearchCV(estimator=svm_model,
                    param_grid=params,
                    cv=5,
                    n_jobs=5,
                    verbose=2)

grid.fit(X_treino_normalized, y_treino)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


(0.8499047619047619, {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'})

In [5]:
predictions = grid.predict(X_teste_normalized)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/svmPrediction.csv')